## Langchain Basic Usage (LLM + ChatModel only)
Langchain的框架下面有主要分為兩種Model
- LLM
- Chat

由於Langchain並不提供任何Model，所以LLM相關的library都不在langchain-core裡面

#### Langchain最基本的用法
`Prompt -> LLM -> Response`

##### Loading environment variables

In [37]:
%load_ext dotenv
%dotenv

import os

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


##### Basic Prompts

In [38]:
# Everything starts with LLM
from langchain_core.messages import HumanMessage
msg = HumanMessage(content='茶餐廳是什麼？')

##### 使用ChatModel (e.g. TogetherAI)

In [39]:
from langchain_together import ChatTogether
model = ChatTogether(
    together_api_key=os.environ['KEY_TOGETHERAI'],
    model="Qwen/Qwen1.5-72B-Chat",
)

In [40]:
res = model.invoke([msg])

##### 利用output_parser將output變得更易讀

In [41]:
print(res)

content='茶餐廳是一種起源于香港的餐飲場所，結合了中西餐飲文化，提供多樣化的菜式和便捷的服務。茶餐廳的菜單通常包括奶茶、菠萝油、西多士、 csak、碟頭飯、炒粉面等飲品和食品，既有中式小吃，也有西式快餐。茶餐廳的特色是食物種類繁多，價格實惠，而且營業時間長，通常全天候供應，是當地人休閒用餐、聚會的常見場所。' response_metadata={'token_usage': {'completion_tokens': 116, 'prompt_tokens': 23, 'total_tokens': 139}, 'model_name': 'Qwen/Qwen1.5-72B-Chat', 'system_fingerprint': None, 'finish_reason': 'eos', 'logprobs': None} id='run-e7bf2310-c388-46a7-b4aa-f7433fcef7be-0' usage_metadata={'input_tokens': 23, 'output_tokens': 116, 'total_tokens': 139}


In [42]:
from langchain_core.output_parsers import StrOutputParser

p = StrOutputParser()
# print(type(p))
print(p.invoke(res))

茶餐廳是一種起源于香港的餐飲場所，結合了中西餐飲文化，提供多樣化的菜式和便捷的服務。茶餐廳的菜單通常包括奶茶、菠萝油、西多士、 csak、碟頭飯、炒粉面等飲品和食品，既有中式小吃，也有西式快餐。茶餐廳的特色是食物種類繁多，價格實惠，而且營業時間長，通常全天候供應，是當地人休閒用餐、聚會的常見場所。


##### 加入SystemMessage (Instruction)

In [43]:
from langchain_core.messages import SystemMessage
system_message = """
必須以繁體中文回答
回答時只以','分隔每個項目，並以最簡短的方式回答，例如：
一,二,三,...
"""
human_message = """
請列舉茶餐廳不多過10種最熱門的食物，不肯定的不要回答
"""
msg_list = [
    SystemMessage(content=system_message),
    HumanMessage(content=human_message)
]

In [44]:
res = model.invoke(msg_list)

In [45]:
print(p.invoke(res))

奶茶,菠蘿油,蛋撻,豬扒飯,魚蛋,炸雞翼,西多士,牛肉窩夫,云吞面,奶茶冰


##### Structured Data Parser
https://python.langchain.com/v0.2/docs/concepts/#output-parsers

In [46]:
from langchain.output_parsers import CommaSeparatedListOutputParser

csvp = CommaSeparatedListOutputParser()
parsed_res = csvp.invoke(res)
print(type(parsed_res))
print(parsed_res)

<class 'list'>
['奶茶', '菠蘿油', '蛋撻', '豬扒飯', '魚蛋', '炸雞翼', '西多士', '牛肉窩夫', '云吞面', '奶茶冰']


##### 用PromptTemplate更方便

In [51]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("human", "請列舉{place}不多過10種最熱門的食物，不肯定的不要回答"),
])

prompt_value = prompt_template.invoke(
    {
        'place': '日本'
    }
)

In [52]:
print(prompt_value)

messages=[SystemMessage(content="\n必須以繁體中文回答\n回答時只以','分隔每個項目，並以最簡短的方式回答，例如：\n一,二,三,...\n"), HumanMessage(content='請列舉日本不多過10種最熱門的食物，不肯定的不要回答')]


In [53]:
japan_csv = csvp.invoke(model.invoke(prompt_value))

In [54]:
print(japan_csv)

['寿司', '拉面', '天妇罗', '烧肉', '章鱼小丸子', '刺身', '乌冬面', '炸鸡', '和牛', '寿喜烧']


##### 以steaming的方式回答

In [58]:
author_system_message = """你是一個日本輕小說作家，寫作風格要模仿{author}，請以用戶的名字命名主角，並以提供的關鍵字去寫一個大約400字的短篇故事"""
prompt_template2 = ChatPromptTemplate.from_messages([
    ("system", author_system_message),
    ("ai", "請問你怎樣稱呼？"),
    ("human", "我的名字叫{username}"),
    ("ai", "你好{username}，請問你想以什麼作題材寫這個故事呢？"),
    ("human", "我想以{topics}為題材寫這個故事"),
])
prompt_value = prompt_template2.invoke(
    {
        "author": "川原礫",
        "username": "彌豆子",
        "topics": "鬼，捉鬼，親情，拯救世界"
    }
)
print(prompt_value)

messages=[SystemMessage(content='你是一個日本輕小說作家，寫作風格要模仿川原礫，請以用戶的名字命名主角，並以提供的關鍵字去寫一個大約400字的短篇故事'), AIMessage(content='請問你怎樣稱呼？'), HumanMessage(content='我的名字叫彌豆子'), AIMessage(content='你好彌豆子，請問你想以什麼作題材寫這個故事呢？'), HumanMessage(content='我想以鬼，捉鬼，親情，拯救世界為題材寫這個故事')]


In [64]:
for res in model.stream(prompt_value):
    print(res.content, end="|")

故事|標|題|：|《||彌|豆|子|的|鬼|之|契|約|》|

|在|一個|被|鬼|怪|支配|的|陰|暗|世界|裡|，|名叫||彌|豆|子|的|少女|，|天生|擁有|能夠|感知|鬼|氣|的|異|能|。|她的|村|莊||屢|遭|鬼|怪|侵|擾|，|村民|生活在|恐||懼|之中|。|自|母親|因|鬼|怪|之|手|離|世|後|，||彌|豆|子|立|下|誓言|要|消|滅|所有|惡|鬼|，|保護|剩下|的人|們|。

|某|日|，|一個|神秘|的|鬼|王|出|現在|村|莊|，|他|提出|一個|契|約|：|若||彌|豆|子|能|成功|捕捉|他|，|他|便|會|幫助|她|拯救|世界|。|沒有|絲|毫|猶|豫|，||彌|豆|子|接受|挑|戰|，|踏|上了|捉|鬼|之旅|。|她|手持|母親|遺|留|的|除|魔|短|刀|，|借助|其|強|大的|力量|，|與|鬼|王|展|開|激|戰|。

|戰|鬥|中|，||彌|豆|子|發現|鬼|王|並|非|純|惡|，|而|是在|某|種|契|約|下|被迫|為|惡|。|她|決定|尋|找|解除|契|約|的方法|，|而非|簡單|地|消|滅|他|。|兩|人的|關係|逐||漸|變化|，|從|敵|對|轉|為|合作|，|一同|尋|求|真相|。

|在|彼此|的|互助|下|，||彌|豆|子|和|鬼|王|揭|開|了|惡|鬼|肆|虐|的|黑暗|陰|謀|，|原來|是|邪|神|在|背|後|操|縱|一切|。|為了|拯救|世界|，|他們| dá|| 出|最後|一|搏|，|用|親|情|與|信任|的力量|，||斬|斷|枷|鎖|，|擊|敗|邪|神|。

|世界|恢|復|了|和平|，||彌|豆|子|與|鬼|王|的|關係|也|成為|了|傳|奇|。|他們|的|勇|氣|和|決|心|，|讓人|們|相信|，|即使是|鬼|，|也有|其|可|貴|之|處|。|而||彌|豆|子|，|成為|了|家喻|戶|曉|的|捉|鬼|英雄|，|她的|故事|鼓舞|著|每|一個|害怕|黑暗|的人|們|，|勇敢|地面|對|生活的|挑|戰|。||

#### LCEL: LangChain Expression Language
怎樣用LCEL簡化日本輕小說家的Chain呢？ 

In [66]:
jnovel = prompt_template2 | model | p
res = jnovel.invoke(
    {
        "author": "支倉凍砂",
        "username": "亨利",
        "topics": "狼，香辛料，北方，旅行商人"
    })

'故事標題：《狼與香辛料的北方旅途》\n\n在遙遠的北方，有一位名叫亨利的旅行商人。他的皮膚因風霜而略顯黝黑，眼神中帶著探索未知的熾烈光芒。亨利不滿足於平凡的交易，他獨自駕駛著獨木舟，穿越凜冽的冰川河流，尋找那些罕見且價值連城的香辛料。\n\n某個風雪交加的夜晚，亨利在森林邊緣的篝火旁歇息，忽然發現一雙琥珀色的眼睛在黑暗中閃爍。一匹狼悄然出現，不同於常見的狼，它的智慧與獨特氣息讓亨利心生敬畏。這匹狼自稱為「赫萝」，是北方的古老精靈，能化作少女形態，她對亨利攜帶的香辛料產生了濃厚的興趣。\n\n亨利與赫萝達成協議，她將成為他的旅伴，以精靈的智慧幫助他尋找更多珍稀香辛料，而亨利則答應帶赫萝見識世界的繁華。於是，這對奇特的搭檔踏上了一段充滿冒險與奇遇的北方旅途。\n\n他們穿越冰封的平原，翻過聳立的山巔，遇見形形色色的人們。赫萝的智慧與亨利的勇氣相輔相成，他們的旅程不僅豐富了亨利的貨物，也緩解了赫萝對家鄉的思念。每一座村落，每一片森林，都留下了他們的足跡和關於香辛料的故事。\n\n在北方的廣袤大地中，亨利與赫萝共同編織著一個關於友情、冒險與香辛料的傳奇，他們的故事成為那些寒冷夜晚中，人們圍繞篝火時最愛分享的佳話。'

In [67]:
print(p.parse(res))

故事標題：《狼與香辛料的北方旅途》

在遙遠的北方，有一位名叫亨利的旅行商人。他的皮膚因風霜而略顯黝黑，眼神中帶著探索未知的熾烈光芒。亨利不滿足於平凡的交易，他獨自駕駛著獨木舟，穿越凜冽的冰川河流，尋找那些罕見且價值連城的香辛料。

某個風雪交加的夜晚，亨利在森林邊緣的篝火旁歇息，忽然發現一雙琥珀色的眼睛在黑暗中閃爍。一匹狼悄然出現，不同於常見的狼，它的智慧與獨特氣息讓亨利心生敬畏。這匹狼自稱為「赫萝」，是北方的古老精靈，能化作少女形態，她對亨利攜帶的香辛料產生了濃厚的興趣。

亨利與赫萝達成協議，她將成為他的旅伴，以精靈的智慧幫助他尋找更多珍稀香辛料，而亨利則答應帶赫萝見識世界的繁華。於是，這對奇特的搭檔踏上了一段充滿冒險與奇遇的北方旅途。

他們穿越冰封的平原，翻過聳立的山巔，遇見形形色色的人們。赫萝的智慧與亨利的勇氣相輔相成，他們的旅程不僅豐富了亨利的貨物，也緩解了赫萝對家鄉的思念。每一座村落，每一片森林，都留下了他們的足跡和關於香辛料的故事。

在北方的廣袤大地中，亨利與赫萝共同編織著一個關於友情、冒險與香辛料的傳奇，他們的故事成為那些寒冷夜晚中，人們圍繞篝火時最愛分享的佳話。


In [73]:
jnovel2 = prompt_template2 | model | p
for res in jnovel2.stream(
    {
        "author": "支倉凍砂",
        "username": "亨利",
        "topics": "狼，香辛料，北方，旅行商人"
    }):
    print(res, end="")

故事名稱：《北方的香辛料冒險》

在遥远的北方，有一个名叫亨利的年轻旅行商人。他的独特之处在于，他不仅贩卖各种珍贵的货物，还拥有一种与狼群沟通的神秘能力。亨利的旅程总是充满了未知与奇遇，他的目标是寻找那些能为人们带来幸福的香辛料。

一日，亨利的商队在暴风雪中迷失了方向，狼群的低吼从远处传来。他通过心灵的桥梁与狼群首领交谈，换取了安全的指引。狼群带领他们穿越了险峻的雪原，找到了一个隐秘的山谷。这里生长着一种罕见的香辛料——"霜雪胡椒"，在寒冷的北方，它能为人们带来温暖与安慰。

在山谷中，亨利遇到了一位名叫艾琳娜的女猎人，她与狼群共生，拥有与亨利相似的天赋。两人结伴而行，共同对抗那些对香辛料图谋不轨的贪婪商人，一起保护这片宁静的山谷。在冒险的旅程中，他们的心逐渐靠近，彼此的默契如同香辛料与食材的融合，温暖而又独特。

随着时间的推移，亨利和艾琳娜的名声在北方传播开来，他们的故事成为了一首赞歌，讲述着勇气、友情与香辛料的力量。他们继续在冰雪覆盖的道路上旅行，为每一个村庄带来北方的祝福，而那霜雪胡椒的香气，也成为了他们冒险旅程的象征。

##### 問題：Print的為何不再是res.content?

https://python.langchain.com/v0.2/docs/how_to/streaming/

### 偉大的LangChain: Runnable
https://python.langchain.com/v0.2/docs/concepts/#runnable-interface